# Amazon Reviews Project

In [1]:
import pandas as pd
import gzip
import os
import json
import scipy as scipy
import ipywidgets as widgets
from IPython.display import display
import statistics as stat
import matplotlib.pyplot as plt
import numpy as np
from sklearn import  linear_model
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

import multiprocessing

## LOAD DATA

In [2]:
category_options={
    'Apps for Android': 'C:/Users/LeTrongTinh/Desktop/reviews_Apps_for_Android_5.json_2.gz',
}
dropdown = widgets.Dropdown(options=category_options)
display(dropdown)
if(not(os.path.isfile(dropdown.value[:-3]))):
    inputFile = gzip.open(dropdown.value, 'rb')
    outputFile = open(dropdown.value[:-3], 'wb')
    outputFile.write(inputFile.read())
    inputFile.close()
    outputFile.close()

In [3]:
data=[]
filepath=dropdown.value[:-3]
with open(filepath) as f:
    for line in f:
        data.append(json.loads(line))
df=pd.DataFrame(data)

In [4]:
print(df.shape)
df.head()

(752937, 9)


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B004A9SDD8,"[1, 1]",3.0,"Loves the song, so he really couldn't wait to ...","11 2, 2013",A1N4O8VOJZTDVB,Annette Yancey,Really cute,1383350400
1,B004A9SDD8,"[0, 0]",5.0,"Oh, how my little grandson loves this app. He'...","12 5, 2011",A2HQWU6HUKIEC7,"Audiobook lover ""Kathy""",2-year-old loves it,1323043200
2,B004A9SDD8,"[0, 0]",5.0,I found this at a perfect time since my daught...,"05 21, 2012",A1SXASF6GYG96I,Barbara Gibbs,Fun game,1337558400
3,B004A9SDD8,"[3, 4]",5.0,My 1 year old goes back to this game over and ...,"12 6, 2012",A2B54P9ZDYH167,"Brooke Greenstreet ""Babylove""",We love our Monkeys!,1354752000
4,B004A9SDD8,"[1, 1]",5.0,There are three different versions of the song...,"02 1, 2014",AFOFZDTX5UC6D,C. Galindo,This is my granddaughters favorite app on my K...,1391212800


In [5]:
train = df.sample(frac=0.8,random_state=200)
test = df.drop(train.index)
print(train.shape, test.shape)

(602350, 9) (150587, 9)


In [6]:
y_train = train['overall']
X_train = train.drop('overall',axis=1)
y_test = test['overall']
X_test = test.drop('overall',axis=1)

In [8]:
X_train.head()

,asin,helpful,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
293554,B00846F1YW,"[0, 0]",This is a fun game. Too much dialogue. Have n...,"02 14, 2013",A3QZ4G18WM9LDX,Vicki Coble Tucker,fun,1360800000
351713,B008L0JFAW,"[4, 4]",I hate this app because I play the cello for s...,"10 28, 2012",A1IMKH5RBJPCJ1,sweetheart555,HORRIBLE,1351382400
413759,B009HKL4B8,"[0, 0]",I like this version of Sims because it does no...,"11 4, 2012",AH4ODNX98K3YA,IAMM3Z,Doesn't harass friends on social media,1351987200
124132,B005ZTFPFQ,"[0, 4]",The reason why this is so crappy because you h...,"12 28, 2013",A1YSTXRBY2R952,caitlin french,Big Mistake.,1388188800
57527,B004UBUK4Q,"[0, 0]",This game is great. Hours of fun and a unique...,"08 21, 2011",AHVB7IR58XW16,"J. Cook ""J Cook""","An excellent, fun challenge! (on hard)",1313884800


In [132]:
X_train["reviewText"].astype(str)

293554    This is a fun game.  Too much dialogue. Have n...
351713    I hate this app because I play the cello for s...
413759    I like this version of Sims because it does no...
124132    The reason why this is so crappy because you h...
57527     This game is great.  Hours of fun and a unique...
647922    This game is easy to pick up just for a few mi...
708472    There is no challenge involved.  Only one who ...
15957     And here I was just talking about break out la...
291575    I have a 7 year old step daughter who has been...
224000    This app works better than the app on my lapto...
65229     I have this on both my phone and kindle. I rec...
344732    This game has a little of everything to offer....
462523    I got this while it was free for the day and i...
497915    This game is the best one so far for me,  espe...
533716    No real point to this app.  Needs to have a po...
281432    This app is a really great app for anyone that...
26380     I've tried to read The Bible b

In [15]:
y_train.head()

293554    4.0
351713    1.0
413759    5.0
124132    1.0
57527     5.0
Name: overall, dtype: float64

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [27]:
text_clf.fit(X_train['reviewText'], y_train)  

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [28]:
from sklearn.metrics import classification_report
predicted = text_clf.predict(X_test['reviewText'])
print(classification_report(y_test, predicted))

             precision    recall  f1-score   support

        1.0       0.79      0.17      0.29     15804
        2.0       0.00      0.00      0.00      8891
        3.0       0.49      0.01      0.01     16956
        4.0       0.34      0.02      0.03     31677
        5.0       0.53      1.00      0.69     77259

avg / total       0.48      0.53      0.39    150587



In [29]:
metrics.confusion_matrix(y_test, predicted)

array([[ 2757,     1,    35,   157, 12854],
       [  419,     0,    63,   258,  8151],
       [  171,     0,   124,   539, 16122],
       [   71,     2,    18,   581, 31005],
       [   88,     8,    13,   163, 76987]])

In [31]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),}

In [32]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [33]:
gs_clf = gs_clf.fit(X_train['reviewText'][:400], y_train[:400])

In [34]:
gs_clf.best_score_ 

0.55500000000000005

In [35]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 2)
